[View in Colaboratory](https://colab.research.google.com/github/kyokicchi/kaggle_codes/blob/master/trainNNmodel.ipynb)

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from keras import models
from keras import layers

from keras.models import load_model


Using TensorFlow backend.


In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%cd ../content/gdrive/My Drive/projects/
%ls

/content/gdrive/My Drive/projects
model_LGB.sav  model_XGB.sav  user_test_n.csv
model_NN.h5    output.csv     user_train_n.csv


In [0]:
s_data = 'user_train_n.csv'
s_col_tgt = 'transactionRevenue_sum'
l_col_drop = [s_col_tgt]
filename = 'model_NN.h5'

In [5]:
%%time
df_train = pd.read_csv(s_data, index_col = 0)

<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 9.02 s, sys: 940 ms, total: 9.96 s
Wall time: 12.8 s


In [6]:
%%time
df_y = df_train[s_col_tgt]
df_x = df_train.drop(l_col_drop, axis=1)

CPU times: user 59.6 ms, sys: 168 ms, total: 228 ms
Wall time: 227 ms


In [0]:
def build_model(N_col):
    model = models.Sequential()
    model.add(layers.Dense(120, activation='relu', input_shape=(N_col,)))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1, activation='relu'))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [11]:
%%time
epochs = 100
batch_size = df_x.shape[0] // 4
validation_split = 0.1

model = build_model(df_x.shape[1])

history = model.fit(df_x,
                    df_y,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split = validation_split,
                    verbose=1)

Train on 642750 samples, validate on 71417 samples
Epoch 1/100
642750/642750 [==============================] - 2s 4us/step - loss: 7.9026e-04 - mean_absolute_error: 0.0043 - val_loss: 2.5418e-05 - val_mean_absolute_error: 4.9837e-04
Epoch 2/100
642750/642750 [==============================] - 1s 2us/step - loss: 2.7971e-05 - mean_absolute_error: 5.1950e-04 - val_loss: 2.5418e-05 - val_mean_absolute_error: 4.9837e-04
Epoch 3/100
642750/642750 [==============================] - 1s 2us/step - loss: 2.7206e-05 - mean_absolute_error: 5.1522e-04 - val_loss: 2.5418e-05 - val_mean_absolute_error: 4.9837e-04
Epoch 4/100
642750/642750 [==============================] - 1s 2us/step - loss: 2.6570e-05 - mean_absolute_error: 5.1056e-04 - val_loss: 2.5418e-05 - val_mean_absolute_error: 4.9837e-04
Epoch 5/100
642750/642750 [==============================] - 1s 2us/step - loss: 2.6564e-05 - mean_absolute_error: 5.0994e-04 - val_loss: 2.5418e-05 - val_mean_absolute_error: 4.9837e-04
Epoch 6/100
642750

In [12]:
history.history

{'loss': [0.0007902634661624355,
  2.7971404500492025e-05,
  2.7206087440977422e-05,
  2.6570200124888942e-05,
  2.656399382752395e-05,
  2.6012542207964492e-05,
  2.633644892841349e-05,
  2.634578154528853e-05,
  2.5977695409320107e-05,
  2.5807666102051516e-05,
  2.5706847611421163e-05,
  2.552410939720623e-05,
  2.5482359780829665e-05,
  2.5438932866929272e-05,
  2.5148902311784938e-05,
  2.51835186429918e-05,
  2.4967380830907743e-05,
  2.5190740569273993e-05,
  2.5026136702065372e-05,
  2.4979181303042274e-05,
  2.495944867081872e-05,
  2.4902459424198615e-05,
  2.4922699868316783e-05,
  2.4896365093582326e-05,
  2.497442532022563e-05,
  2.491358928501993e-05,
  2.489937077594072e-05,
  2.489081690112445e-05,
  2.4948866061883896e-05,
  2.4903227975334938e-05,
  2.4887276900973875e-05,
  2.4887406903219717e-05,
  2.4901679196805874e-05,
  2.4889139433519806e-05,
  2.4888643788008342e-05,
  2.4887176481984948e-05,
  2.488717678854586e-05,
  2.4897577653859803e-05,
  2.4887175776610

In [13]:
%%time
model.save(filename)

#files.download(filename)

CPU times: user 46.7 ms, sys: 6.95 ms, total: 53.7 ms
Wall time: 459 ms
